In [2]:
import dask.array as da
import numpy as np
from dask.dot import dot_graph
#from dask.multiprocessing import get
from dask import get
from functools import partial
from time import sleep, clock
from scipy.integrate import simps
from scipy.special import gamma

from dask.callbacks import Callback
#from multiprocessing.pool import ThreadPool

from ForwardEquation import forwardEquation, forwardEquation_dask
from Gradient import computeGradient, computeGradient_dask
from TricubicInterpolation import TriCubic
from LineSearch import lineSearch
from InfoCompleteness import precondition
from InitialModel import createInitialModel
from CalcRays import calcRays,calcRays_dask
from RealData import plotDataPack
from PlotTools import animateTCISlices


outputfolder = 'output/test/MH_5'



def store_forwardEq(n,templateDatapack,antIdx,timeIdx,dirIdx,rays,K_ne,mTCI,i0):
    filename = "{}/g_{}.hdf5".format(outputfolder,n)
    if os.path.isfile(filename):
        return filename
    g = forwardEquation_dask(rays,K_ne,mTCI,i0)
    datapack = templateDatapack.clone()
    datapack.set_dtec(g,antIdx=antIdx,timeIdx=timeIdx,dirIdx = dirIdx)
    datapack.save(filename)
    dobs = templateDatapack.get_dtec(antIdx=antIdx,timeIdx=timeIdx,dirIdx = dirIdx)
    vmin = np.min(dobs)
    vmax = np.max(dobs)
    plotDataPack(datapack,antIdx=antIdx,timeIdx=timeIdx,dirIdx = dirIdx,
                figname=filename.split('.')[0], vmin = vmin, vmax = vmax)#replace('hdf5','png'))
    return filename

def pull_forwardEq(filename,antIdx,timeIdx,dirIdx):
    import pylab as plt
    datapack = DataPack(filename=filename)
    g = datapack.get_dtec(antIdx=antIdx,timeIdx=timeIdx,dirIdx = dirIdx)
    return g

def negloglike(g,dobs,CdCt):
    L1 = g-dobs
    L1 *= L1
    #np.abs(L2,out=L2)
    #L2 = np.power(L2,2.,out=L2)
    L1 /= CdCt
    np.sqrt(L1,out=L1)
    return np.sum(L1)
    #return np.sum(L2)/2.
def likelihood(g,dobs,CdCt):
    #print(g,dobs,g-dobs)
    neglog = negloglike(g,dobs,CdCt)
    return np.exp(-neglog)

def metropolisHastings(binning,datapack,L_ne,sizeCell,i0, antIdx=-1, dirIdx=-1, timeIdx = [0]):
    print("Using output folder: {}".format(outputfolder))
    import pylab as plt
    straightLineApprox = True
    tmax = 1000.
    antennas,antennaLabels = datapack.get_antennas(antIdx = antIdx)
    patches, patchNames = datapack.get_directions(dirIdx = dirIdx)
    times,timestamps = datapack.get_times(timeIdx=timeIdx)
    datapack.setReferenceAntenna(antennaLabels[i0])
    #plotDataPack(datapack,antIdx = antIdx, timeIdx = timeIdx, dirIdx = dirIdx,figname='{}/dobs'.format(outputfolder))
    dobs = datapack.get_dtec(antIdx = antIdx, timeIdx = timeIdx, dirIdx = dirIdx)
    Na = len(antennas)
    Nt = len(times)
    Nd = len(patches) 
    fixtime = times[Nt>>1]
    phase = datapack.getCenterDirection()
    arrayCenter = datapack.radioArray.getCenter()
    #Average time axis down and center on fixtime
    if Nt == 1:
        var = (0.5*np.percentile(dobs[dobs>0],25) + 0.5*np.percentile(-dobs[dobs<0],25))**2
        CdCt = np.ones([Na,1,Nd],dtype=np.double)*var
    else:
        dt = times[1].gps - times[0].gps
        print("Averaging down window of length {} seconds [{} timestamps]".format(dt*Nt, Nt))
        CdCt = np.stack([np.var(dobs,axis=1)],axis=1) 
        dobs = np.stack([np.mean(dobs,axis=1)],axis=1)
        timeIdx = [Nt>>1]
        times,timestamps = datapack.get_times(timeIdx=timeIdx)
        Nt = len(times)
    #CdCt = np.ones([Na,1,Nd],dtype=np.double)*0.01**2
    CdCt += 1e-15
    print("CdCt: {} +- {}".format(np.mean(CdCt), np.std(CdCt)))
    vmin = np.min(datapack.get_dtec(antIdx = antIdx, timeIdx = timeIdx, dirIdx = dirIdx))
    vmax = np.max(datapack.get_dtec(antIdx = antIdx, timeIdx = timeIdx, dirIdx = dirIdx))
    plotDataPack(datapack,antIdx=antIdx,timeIdx=timeIdx,dirIdx = dirIdx,
            figname='{}/dobs'.format(outputfolder), vmin = vmin, vmax = vmax)#replace('hdf5','png'))
    #createPrior
    neTCI = createInitialModel(datapack,antIdx = antIdx, timeIdx = timeIdx, dirIdx = dirIdx, zmax = tmax,spacing=sizeCell)
    neTCI.save("{}/nePriori.hdf5".format(outputfolder))
    rays = calcRays(antennas,patches,times, arrayCenter, fixtime, phase, neTCI, datapack.radioArray.frequency, 
                    straightLineApprox, tmax, neTCI.nz)
    K_ne = np.mean(neTCI.m)
    mTCI = neTCI.copy()
    mTCI.m /= K_ne
    np.log(mTCI.m,out=mTCI.m)
    templateDatapack = datapack.clone()
    i = 1
    accepted = 0
    Nmax = int(1e3)
    binning = 10
    #1e9 is a GB
    mAccepted = np.zeros([mTCI.m.size,Nmax],dtype=np.double)
    m_i = mTCI.m.copy()
    mAccepted[:,0] = m_i
    mML = mTCI.m.copy()
    g = forwardEquation_dask(rays,K_ne,mTCI,i0)
    Si = negloglike(g,dobs,CdCt)
    Li = np.exp(-Si)
    maxL = Li
    #sampling
    lvec = (np.fft.fftfreq(mTCI.nx,d=mTCI.xvec[1]-mTCI.xvec[0]))
    mvec = (np.fft.fftfreq(mTCI.ny,d=mTCI.yvec[1]-mTCI.yvec[0]))
    nvec = (np.fft.fftfreq(mTCI.nz,d=mTCI.zvec[1]-mTCI.zvec[0]))
    L_,M_,N_ = np.meshgrid(lvec,mvec,nvec,indexing='ij')
    R2 = L_**2 + M_**2 + N_**2
    theta1 = np.log(2/0.75)/2.
    theta2 = L_ne
    theta3 = 5./2.
    omega = theta1*theta2**3/gamma(theta3) /np.pi**(3./2.) * (1. + theta2**2 *R2)**(-(theta3 + 3./2.))
    np.sqrt(omega,out=omega)
    V = (mTCI.xvec[-1]-mTCI.xvec[0])*(mTCI.yvec[-1]-mTCI.yvec[0])*(mTCI.zvec[-1]-mTCI.zvec[0])
    omega /= V
    Ncycle = 10
    while accepted < binning**2 and i < Nmax:
        cycle = 1.05 + 1.2*np.cos((i%Ncycle)*np.pi/(Ncycle-1)/2.)
        theta1 = np.log(cycle)
        dM = np.fft.fftn(np.random.normal(size=L_.shape))
        dM *= omega
        dM = np.fft.ifftn(dM).real.ravel('C')
        dM *= theta1/np.max(dM)
        mTCI.m += dM
        g = forwardEquation_dask(rays,K_ne,mTCI,i0)
        Sj = negloglike(g,dobs,CdCt)
        Lj = np.exp(-Sj)
        if Sj < Si or np.log(np.random.uniform()) < Si - Sj:
            mAccepted[:,i] += mTCI.m
            print("Accepted, Si {}, Sj {}".format(Si,Sj))
            templateDatapack.set_dtec(g,antIdx=antIdx,timeIdx=timeIdx,dirIdx = dirIdx)
            templateDatapack.save("{}/g{}.hdf5".format(outputfolder,accepted))
            plotDataPack(templateDatapack,antIdx=antIdx,timeIdx=timeIdx,dirIdx = dirIdx,
            figname='{}/g{}'.format(outputfolder,accepted), vmin = vmin, vmax = vmax)#replace('hdf5','png'))
            np.exp(mTCI.m, out=neTCI.m)
            neTCI.m *= K_ne
            neTCI.save("{}/m{}.hdf5".format(outputfolder,accepted))
            Si = Sj
            Li = Lj
            accepted += 1
        else:
            mAccepted[:,i] += mAccepted[:,i-1]
            print("Rejected, Si {}, Sj {}".format(Si,Sj))
            mTCI.m -= dM
        if Lj > maxL:
            maxL = Lj
            print("New maxL: {}".format(maxL))
            mML = mTCI.m.copy()
        i += 1
    if accepted == binning**2:
        print("Converged in {} steps".format(i))
    print("Acceptance: {}, rate : {}".format(accepted,float(accepted)/i))
    mAccepted = mAccepted[:,:i]
    meanModel = np.mean(mAccepted,axis=1)
    mTCI.m = meanModel
    np.exp(mTCI.m, out=neTCI.m)
    neTCI.m *= K_ne
    neTCI.save("{}/meanModel.hdf5".format(outputfolder))
    g = forwardEquation_dask(rays,K_ne,mTCI,i0)
    templateDatapack.set_dtec(g,antIdx=antIdx,timeIdx=timeIdx,dirIdx = dirIdx)
    templateDatapack.save("{}/post_g.hdf5".format(outputfolder))
    plotDataPack(templateDatapack,antIdx=antIdx,timeIdx=timeIdx,dirIdx = dirIdx,
    figname='{}/post_g'.format(outputfolder), vmin = vmin, vmax = vmax)#replace('hdf5','png'))
    #animateTCISlices(mTCI,"{}/meanModelPlot".format(outputfolder),numSeconds=20.)
    
    #covariance
    varModel = np.var(mAccepted,axis=1)
    mTCI.m = varModel
    mTCI.save("{}/varModel.hdf5".format(outputfolder))
    #animateTCISlices(mTCI,"{}/varModelPlot".format(outputfolder),numSeconds=20.)
    
    
if __name__ == '__main__':
    import os
    try:
        os.makedirs(outputfolder)
    except:
        pass
    from RealData import DataPack
    from AntennaFacetSelection import selectAntennaFacets
    #from InitialModel import createTurbulentlModel
    i0 = 0
    datapack = DataPack(filename="output/test/datapackObs.hdf5")
    #flags = datapack.findFlaggedAntennas()
    #datapack.flagAntennas(flags)
    datapackSel = selectAntennaFacets(16, datapack, antIdx=-1, dirIdx=-1, timeIdx = np.arange(4))
    #pertTCI = createTurbulentlModel(datapackSel,antIdx = -1, timeIdx = -1, dirIdx = -1, zmax = 1000.)
    L_ne = 50.
    sizeCell = 5.
    
    metropolisHastings(25,datapackSel,L_ne,sizeCell,i0, antIdx=-1, dirIdx=-1, timeIdx = np.arange(4))  

Setting refAnt: CS201HBA1
Loaded 15 antennas, 3595 times, 15 directions
Setting refAnt: CS201HBA1
flagged ['CS201HBA1', 'RS406HBA', 'RS307HBA', 'RS106HBA', 'CS103HBA0', 'CS103HBA1', 'RS205HBA']
Setting refAnt: CS401HBA1
Loaded 8 antennas, 3595 times, 15 directions
Setting refAnt: CS401HBA1
flagged ['s1', 's253', 's27', 's26', 's281', 's18', 's21']
Using radio array Radio Array: 1.50000e+08 MHz, Longitude 6.78 deg Latitude 52.91 deg Height -21.88 m
Using phase center 217.76998437499998 deg 34.85346701388889 deg
Fixing frame at 2014-08-10T13:00:00.000
Elevation is 46.24504165116317 deg
Sun at zenith angle 40.708514731932155
Creating ionosphere model...
Found domain u in -192.32037971297675..185.2934901454661, v in -207.6767568035685..195.59064744183414, w in -129.72929514485605..1107.8154421757695
Nx=76 Ny=81 Nz=248 number of cells: 1526688
plotting levels : [1.5870235234806583e-12, 2.0716257317089288e-12, 4.0486658597235683e-12, 6.5529893996088037e-12, 1.0248554262241074e-11, 1.59750497

C:\Users\josh\Anaconda3\envs\compute\lib\site-packages\matplotlib\contour.py:370: RuntimeWarning: invalid value encountered in true_divide
  dist = np.add.reduce(([(abs(s)[i] / L[i]) for i in range(xsize)]), -1)
